## Packages

In [1]:
library(here)
library(tidyverse)
library(stringr)
library(readr)
library(writexl)
library("stopwords")
devtools::load_all("~/Code/R/ktools/")

here() starts at /Users/knguyen/Documents/WIP/Papers/Astrid

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.2 ──
✔ tibble  3.1.8      ✔ dplyr   1.0.10
✔ tidyr   1.2.1      ✔ stringr 1.5.0 
✔ readr   2.1.3      ✔ forcats 0.5.2 
✔ purrr   1.0.1      
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
Finding a python executable with spaCy installed...

spaCy (language model: en_core_web_trf) is installed in more than one python

spacyr will use /Users/knguyen/opt/miniconda3/bin/python (because ask = FALSE)

successfully initialized (spaCy Version: 3.3.1, language model: en_core_web_trf)

(python options: type = "python_executable", value = "/Users/knguyen/opt/miniconda3/bin/python")


Attaching package: ‘magrittr’


The following object is masked from ‘package:purrr’:

    set_names


The following object is masked from ‘package:tidyr’:

    extract


ℹ 

── R CMD INSTALL ───────────────────────────────────────────────────────────────
* installing *source* package ‘ktools’ ...
** using staged installation
** libs
* DONE (ktools)


Package version: 3.2.3
Unicode version: 14.0
ICU version: 71.1

Parallel computing: 8 of 8 threads used.

See https://quanteda.io for tutorials and examples.



Only needed for German

In [2]:
Sys.getlocale()
Sys.setlocale("LC_ALL", "de_DE.UTF-8")
Sys.getlocale()

[1] "C/UTF-8/C/C/C/C"

[1] "de_DE.UTF-8/de_DE.UTF-8/de_DE.UTF-8/C/de_DE.UTF-8/C"

[1] "de_DE.UTF-8/de_DE.UTF-8/de_DE.UTF-8/C/de_DE.UTF-8/C"

## Forum data

Combine forum texts

In [ ]:
forum <- purrr::map_dfr(
    list.files("~/Documents/WIP/Papers/Astrid/data/new/", full.names = 1), 
    read_csv, col_select = -1, show_col_types = FALSE
)

Rearrange by post ID

In [12]:
forum %<>% 
    mutate(id = gsub("post_message_(.*)", "\\1", mid)) %>%
    arrange(id)

### Emoji

List of emoji [from the forum](https://bfriends.brigitte.de/foren/misc.php).

In [384]:
emoji_ls <- read_csv('https://raw.githubusercontent.com/kklot/netnography/main/emoticons.csv', show_col_types = FALSE)
emoji_ls %>% glimpse

Rows: 69
Columns: 3
$ `:smile:` <chr> ":ooooh:", ":kuss:", ":lachen:", ":peinlich:", ":niedergesch…
$ Smile     <chr> "Ooooh", "Kuss", "Lachen", "Peinlich", "Niedergeschmettert",…
$ smile     <chr> "ooooh", "kiss", "laugh", "embarrassed", "crushed", "offende…


Emoji recode to UTF-8

In [209]:
emj_try <- function(x) {
    o <- tryCatch(emoji::emoji(x), error = function(e) e)
    if (inherits(o, "error")) o <- "TBD"
    o
}

In [329]:
emoji_ls %>% 
rename_with(~char(shortcode, title, meaning)) %>%
select(-title) %>%
mutate(
    meaning = str_replace_all(meaning, '\\s', '_'),
    meaning = case_when(
        meaning == "kiss" ~ 'kissing_face_with_closed_eyes', 
        meaning == "ooooh" ~ 'astonished', 
        meaning == "embarrassed" ~ 'flushed', 
        meaning == "crushed" ~ 'anguished_face', 
        meaning == "offended" ~ 'unamused_face', 
        meaning == "grump" ~ 'angry', 
        meaning == "smack" ~ 'face_with_diagonal_mouth', 
        meaning == "cool" ~ 'smiling_face_with_sunglasses', 
        meaning == "smirksmile" ~ 'smirking_face', 
        meaning == "flower" ~ 'tulip', 
        meaning == "cheeky_grin" ~ 'beaming_face_with_smiling_eyes', 
        meaning == "submission" ~ 'roll_eyes', 
        meaning == "ahoy" ~ 'waving_hand', 
        meaning == "cheer" ~ 'clinking_glasses', 
        meaning == "everything_o.k." ~ 'ok_hand', 
        meaning == "musician" ~ 'partying_face', 
        meaning == "yeah" ~ 'stuck_out_tongue', 
        meaning == "santa_claus" ~ 'santa', 
        meaning == "snow_santa_claus" ~ 'santa', 
        meaning == "sun" ~ 'sun_with_face', 
        meaning == "crown_on" ~ 'person_with_crown', 
        meaning == "engel" ~ 'smiling_face_with_halo', 
        meaning == "against_the_wall" ~ 'person_facepalming', 
        meaning == "cheek_kiss" ~ 'kissing_face_with_smiling_eyes', 
        meaning == "relax" ~ 'smiling_face', 
        meaning == "book_reading" ~ 'open_book', 
        meaning == "embrace" ~ 'people_hugging', 
        meaning == "how." ~ 'confused_face', 
        meaning == "toothache" ~ 'face_with_head_bandage', 
        meaning == "hunger" ~ 'face_savoring_food', 
        meaning == "white_candle" ~ 'candle', 
        meaning == "whistling" ~ 'kissing_face', # double meaning emoji
        meaning == "sleep_well" ~ 'sleeping_face',
        meaning == "pc_broken" ~ 'angry_face',
        meaning == "easter_miley" ~ 'hatching_chick',
        meaning == "easter_bunny" ~ 'rabbit_face',
        meaning == "sign_no" ~ 'thumbs_down',
        meaning == "sign_exactly" ~ 'thumbs_up',
        meaning == "fluestertuete" ~ 'loudspeaker',
        meaning == "rolling_pin" ~ 'pouting_face',
        meaning == "aetsch" ~ 'squinting_face_with_tongue',
        meaning == "cheeky_grinning_angel" ~ 'beaming_face_with_smiling_eyes',
        meaning == "football_girl" ~ 'soccer_ball',
        meaning == "laughing_tears" ~ 'face_with_tears_of_joy',
        meaning == "knitting" ~ 'yarn',
        meaning == "wipe_sweat_from_forehead" ~ 'anxious_face_with_sweat',
        meaning == "sign_...oops" ~ 'face_with_hand_over_mouth',
        meaning == "cup_clown" ~ 'hot_beverage',
        meaning == "enlightenment" ~ 'light_bulb',
        meaning == "flower_gift" ~ 'gift',
        meaning == "knicks" ~ 'woman_bowing',
        otherwise ~ meaning
    ),
    emoji = map_chr(meaning, emj_try)
) %>% 
allot(emoji_converted)

Add one missing (quite strange)

In [394]:
emoji_converted %<>% 
bind_rows(
    tibble(
    shortcode = c(":joy:", ":persevere:", ":dankbar:"), 
    meaning = c("joy", "persevere", "thankful"), 
    emoji = c(emoji::emoji('joy'), emoji::emoji('persevere'), emoji::emoji('folded_hands'))
)) %>% distinct()

write_csv(emoji_converted, 'emoji_converted.csv')

Replace shortcode with emoji

In [364]:
used_emj <- lapply(forum$text, str_match_all, pattern = ':[a-z]*:') %>% 
unlist %>% unique %>% str_subset('^N|::', negate = T)
used_emj

[1] ":zwinker:"            ":knatsch:"            ":ooooh:"             
 [4] ":blumengabe:"         ":unterwerf:"          ":smirksmile:"        
 [7] ":liebe:"              ":pfeifend:"           ":zauberer:"          
[10] ":jubel:"              ":erleuchtung:"        ":smile:"             
[13] ":heul:"               ":niedergeschmettert:" ":beleidigt:"         
[16] ":engel:"              ":yeah:"               ":peinlich:"          
[19] ":blume:"              ":aetsch:"             ":lachen:"            
[22] ":grmpf:"              ":sonne:"              ":teufelchen:"        
[25] ":wangenkuss:"         ":zahnschmerzen:"      ":nudelholz:"         
[28] ":krank:"              ":kristallkugel:"      ":regen:"             
[31] ":schneemann:"         ":joy:"                ":ahoi:"              
[34] ":musiker:"            ":persevere:"          ":kuss:"              
[37] ":halloween:"          ":tassenkasper:"       ":cool:"              
[40] ":osterhase:"          ":prost:"              ":hund:"              
[43] ":kerze:"              ":nikolaus:"           ":entspann:"          
[46] ":knicks:"             ":hunger:"             ":fluestertuete:"     
[49] ":dankbar:"

In [395]:
forum$text2 <- forum$text
for (i in 1:length(used_emj)) {
    id <- which(tolower(emoji_converted$shortcode) == used_emj[i])
    cat('Try ', used_emj[i], emoji_converted$emoji[id], "\r")
    flush.console()
    forum$text2 <- sapply(
        forum$text2, 
        str_replace_all, 
        pattern = used_emj[i], 
        replacement = emoji_converted$emoji[id]
        )
}

In [399]:
head(forum, 3)

user,date,time,mid,text,user_quoted,quoted_text,id,text2
<chr>,<chr>,<time>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
BRIGITTE Community-Team,"02.06.2020,",13:24:00,post_message_34705014,"Liebe User*innen, neben Fragen der Kinderbetreuung, Kitas oder Home-Schooling soll hier auch Platz für weitere Familienthemen wie z.B. den Umgang mit An gehörigen in Seniorenheimen sein. Hier können Sie sehen, welchen bisherigen Themen in diesem neuen Thema aufgehen Wir wünschen Ihnen spannende Diskussionen. Viele Grüße vom BRIGITTE Community-Team",NA,NA,34705014,"Liebe User*innen, neben Fragen der Kinderbetreuung, Kitas oder Home-Schooling soll hier auch Platz für weitere Familienthemen wie z.B. den Umgang mit An gehörigen in Seniorenheimen sein. Hier können Sie sehen, welchen bisherigen Themen in diesem neuen Thema aufgehen Wir wünschen Ihnen spannende Diskussionen. Viele Grüße vom BRIGITTE Community-Team"
BRIGITTE Community-Team,"02.06.2020,",15:19:00,post_message_34705172,"Da das Thema durch den Umbau weiter nach hinten gerutscht ist, schieben wir es auf diesem Weg wieder nach vorne. Wir wünschen Ihnen spannende Diskussionen. Viele Grüße vom BRIGITTE Community-Team",NA,NA,34705172,"Da das Thema durch den Umbau weiter nach hinten gerutscht ist, schieben wir es auf diesem Weg wieder nach vorne. Wir wünschen Ihnen spannende Diskussionen. Viele Grüße vom BRIGITTE Community-Team"
Inaktiver User,"02.06.2020,",18:29:00,post_message_34705352,"Vielen Dank, liebes Community-Team. Es ist so wie eine ""Reinigung"" des Threads ... vielleicht kommen jetzt neue Fragen, Sorgen, Lösungen, Ideen auf ... manchmal ist weniger, mehr ... :zwinker:",NA,NA,34705352,"Vielen Dank, liebes Community-Team. Es ist so wie eine ""Reinigung"" des Threads ... vielleicht kommen jetzt neue Fragen, Sorgen, Lösungen, Ideen auf ... manchmal ist weniger, mehr ... 😉"


Remove deleted messages

In [406]:
forum %<>% filter(!is.na(mid))

In [413]:
forum %<>%
    mutate(mid = gsub('post_message_', '', mid)) %>% 
    arrange(mid)

In [407]:
saveRDS(forum, here('data/forum_emj.rds'))